In [ ]:
# Conversational Context: Building Stateful Agents
# Welcome 👋🏻 In this notebook, you’ll learn how to leverage conversational context (state)
# within Google’s Agent Development Kit (ADK) to build more intelligent and natural agents.
# This is crucial for multi-turn conversations where the agent needs to remember
# information from previous interactions (e.g., the last city mentioned, a user preference)
# to provide relevant responses without constant re-specification.

# We will enhance our Weather Agent by making its custom tools "stateful," allowing them
# to read from and write to the session's `tool_context.state`.

In [ ]:
# Install Google ADK for Python
# This foundational package provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

In [ ]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

In [ ]:
# Configure environment
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Load API key from .env file
# Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here
api_key = os.environ.get("GOOGLE_GENAI_API_KEY")

if not api_key:
    raise ValueError(
        "GOOGLE_GENAI_API_KEY not found.\n"
        "Create a .env file with: GOOGLE_GENAI_API_KEY=your-api-key-here\n"
        "Get your API key from: https://aistudio.google.com/apikey"
    )

os.environ["GEMINI_API_KEY"] = api_key

print(f"✓ API key configured (length: {len(api_key)} characters)")
print(f"✓ GOOGLE_GENAI_USE_VERTEXAI = {os.environ.get('GOOGLE_GENAI_USE_VERTEXAI')}")

# Define the specific Gemini model we'll use.
# 'gemini-2.0-flash' is a fast and efficient model suitable for many tasks.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [ ]:
# Import Necessary Modules
import requests # Used for making HTTP requests to external APIs (OpenWeatherMap)
from typing import Dict, Any, Optional # Type hints for better code readability and error checking
from datetime import datetime, timedelta, timezone # For handling dates and time calculations and timezone objects
from collections import Counter # For counting occurrences of weather descriptions
from dateutil import parser as date_parser # A robust library for parsing various date strings
import calendar # For calendar-related calculations, e.g., finding weeks in a month
import re # Regular expressions for parsing complex date expressions
import time # For adding delays to avoid hitting API rate limits during testing

# ADK Core Modules
from google.adk.agents import Agent # The fundamental class for creating an AI agent
from google.adk.runners import Runner # The orchestrator that manages agent execution
from google.adk.sessions import InMemorySessionService # Simple, in-memory session management
from google.adk.memory import InMemoryMemoryService # For managing longer-term agent memory (though for this notebook, state is primarily handled by tool_context.state)
from google.genai import types # Data structures (like Content and Part) for LLM interaction
from IPython.display import Markdown, display # For rendering rich text output in notebooks

In [ ]:
# OpenWeather API: Set Up
# It's crucial to set your OpenWeatherMap API key as an environment variable for security and ease of management.
# You can obtain a free API key from https://openweathermap.org/api
OPEN_WEATHER_API_KEY = os.environ.get("OPEN_WEATHER_API_KEY")

# Raise an error if the API key is not set, preventing execution without proper authentication.
if not OPEN_WEATHER_API_KEY:
    raise ValueError(
        "OPEN_WEATHER_API_KEY environment variable not set. Please set it. "
        "Get your key from https://openweathermap.org/api and run 'export OPEN_WEATHER_API_KEY=\"YOUR_KEY_HERE\"' "
        "in your terminal, or add it to your .env file if using python-dotenv."
    )

print("OpenWeather API key loaded (if set).")

In [ ]:
# Helper Functions: Parse flexible date expressions for weather queries
# This function is designed to interpret a wide range of natural language date requests
# (e.g., "tomorrow", "next week", "first week of July") and convert them into precise
# start and end datetime objects. This makes the weather agent much more user-friendly.

def parse_flexible_date_range(date_str: str) -> Optional[tuple]:
    """
    Parses a wide variety of date expressions and returns (start_date, end_date).
    Supports:
    - 'today', 'tomorrow', 'in 3 days'
    - 'this weekend', 'next weekend'
    - 'next week', 'this week', 'last week'
    - 'first week of July', 'second week of August', etc.
    - Specific dates: '2025-07-01', 'July 1', '07-01'
    - Date ranges: 'July 1 to July 5', 'next 3 days'
    Returns (datetime, datetime) or None if parsing fails.
    """
    date_str = date_str.lower().strip()
    today = datetime.today()
    weekday = today.weekday()  # Monday=0, Sunday=6

    # Handle 'today', 'tomorrow', 'in X days'
    target_date = None
    if date_str in ["today", "now"]:
        target_date = today
    elif date_str == "tomorrow":
        target_date = today + timedelta(days=1)
    
    match = re.match(r"in (\d+) days?", date_str)
    if match:
        days = int(match.group(1))
        target_date = today + timedelta(days=days)

    if target_date:
        # For single day targets, set range from start of day to end of day
        start_of_day = target_date.replace(hour=0, minute=0, second=0, microsecond=0)
        end_of_day = target_date.replace(hour=23, minute=59, second=59, microsecond=999999)
        return start_of_day, end_of_day
    
    # Handle "next X days" or "for the next X days"
    match = re.match(r"(?:next|for the next) (\d+) days?", date_str)
    if match:
        days = int(match.group(1))
        start = today.replace(hour=0, minute=0, second=0, microsecond=0) # Start of today
        end = today + timedelta(days=days - 1)
        end = end.replace(hour=23, minute=59, second=59, microsecond=999999) # End of the last day
        return start, end

    # Handle 'this week' (Monday to Sunday)
    if date_str == "this week":
        start = today - timedelta(days=weekday)
        end = start + timedelta(days=6)
        # Set to start/end of day for clear boundaries
        start = start.replace(hour=0, minute=0, second=0, microsecond=0)
        end = end.replace(hour=23, minute=59, second=59, microsecond=999999)
        return start, end

    # Handle 'next week' (Monday to Sunday)
    if date_str == "next week":
        start = today - timedelta(days=weekday) + timedelta(days=7)
        end = start + timedelta(days=6)
        # Set to start/end of day for clear boundaries
        start = start.replace(hour=0, minute=0, second=0, microsecond=0)
        end = end.replace(hour=23, minute=59, second=59, microsecond=999999)
        return start, end

    # NEW: Handle 'last week' (Monday to Sunday)
    if date_str == "last week":
        start = today - timedelta(days=weekday) - timedelta(days=7)
        end = start + timedelta(days=6)
        start = start.replace(hour=0, minute=0, second=0, microsecond=0)
        end = end.replace(hour=23, minute=59, second=59, microsecond=999999)
        return start, end

    # This weekend (Saturday & Sunday)
    if date_str == "this weekend":
        saturday = today + timedelta((5 - weekday) % 7)
        sunday = saturday + timedelta(days=1)
        saturday = saturday.replace(hour=0, minute=0, second=0, microsecond=0)
        sunday = sunday.replace(hour=23, minute=59, second=59, microsecond=999999)
        return saturday, sunday

    # Next weekend (Saturday & Sunday)
    if date_str == "next weekend":
        saturday = today + timedelta((5 - weekday) % 7 + 7)
        sunday = saturday + timedelta(days=1)
        saturday = saturday.replace(hour=0, minute=0, second=0, microsecond=0)
        sunday = sunday.replace(hour=23, minute=59, second=59, microsecond=999999)
        return saturday, sunday

    # "first week of July", "second week of August", etc.
    match = re.match(r"(first|second|third|fourth|last) week of (\\w+)", date_str)
    if match:
        week_map = {
            "first": 0, "second": 1, "third": 2, "fourth": 3, "last": -1
        }
        week_num = week_map[match.group(1)]
        month_str = match.group(2)
        try:
            month = list(calendar.month_name).index(month_str.capitalize())
            year = today.year
            if month < today.month:
                year += 1
            cal = calendar.monthcalendar(year, month)
            
            if week_num == -1:
                week = cal[-1]
            elif week_num < len(cal):
                week = cal[week_num]
            else:
                return None
            start_day_of_week = None
            end_day_of_week = None
            for d in week:
                if d != 0:
                    if start_day_of_week is None:
                        start_day_of_week = d
                    end_day_of_week = d
            if start_day_of_week is not None and end_day_of_week is not None:
                start = datetime(year, month, start_day_of_week).replace(hour=0, minute=0, second=0, microsecond=0)
                end = datetime(year, month, end_day_of_week).replace(hour=23, minute=59, second=59, microsecond=999999)
                return start, end
            else:
                return None
        except ValueError:
            return None
        except Exception:
            return None

    # Try parsing as a specific date or date range
    try:
        dt = date_parser.parse(date_str, fuzzy=True, default=today)
        start = dt.replace(hour=0, minute=0, second=0, microsecond=0)
        end = dt.replace(hour=23, minute=59, second=59, microsecond=999999)
        return start, end
    except Exception:
        pass

    if " to " in date_str:
        parts = date_str.split(" to ")
        try:
            start = date_parser.parse(parts[0], fuzzy=True, default=today).replace(hour=0, minute=0, second=0, microsecond=0)
            end = date_parser.parse(parts[1], fuzzy=True, default=today).replace(hour=23, minute=59, second=59, microsecond=999999)
            return start, end
        except Exception:
            return None

    return None

print("`parse_flexible_date_range` helper function defined.")

In [ ]:
# Custom Tool: Get Current Weather (stateful version)
# This function is a custom tool that the agent can call. It fetches real-time
# weather data for a specified city using the OpenWeatherMap Current Weather API.
# It leverages `tool_context.state` to:
# 1. Read `user_preference_temperature_unit` to determine Celsius or Fahrenheit.
# 2. If no city is explicitly provided in the tool call, it attempts to use `last_city_checked_stateful`
#    from the session state.
# 3. Writes the `city` it successfully checked into `last_city_checked_stateful` for future turns.

def get_current_weather_from_openweather_stateful(city: Optional[str], tool_context) -> Dict[str, Any]:
    """
    Retrieves current weather data for a given city from OpenWeatherMap,
    considering user preferences and session state for context.
    Updates last_city_checked_stateful regardless of API success/failure.
    """
    # Get user's preferred temperature unit from session state, defaulting to metric
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "metric")
    unit_symbol = "°C" if preferred_unit == "metric" else "°F"

    # If no city is provided, try to get it from the last checked city in session state
    if not city:
        city = tool_context.state.get("last_city_checked_stateful")
        if not city: # If still no city, return an error
            return {"status": "error", "error_message": "No city specified or remembered for current weather. Please provide a city."}

    print(f"--- Tool: get_current_weather_from_openweather_stateful called for city: {city}, unit: {preferred_unit} ---")
    
    response_data = {} # Initialize to store tool response
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPEN_WEATHER_API_KEY}&units={preferred_unit}"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data["cod"] == 200:
            description = data["weather"][0]["description"]
            temperature = data["main"]["temp"]
            feels_like = data["main"]["feels_like"]
            humidity = data["main"]["humidity"]
            wind_speed = data["wind"]["speed"]
            report = (
                f"The current weather in {city} is {description} with a temperature of {temperature}{unit_symbol} "
                f"(feels like {feels_like}{unit_symbol}). Humidity is {humidity}% and wind speed is {wind_speed} m/s."
            )
            response_data = {"status": "success", "report": report}
        else:
            response_data = {"status": "error", "error_message": f"OpenWeatherMap API Error: {data.get('message', 'Unknown error')}"}
    except requests.exceptions.Timeout:
        response_data = {"status": "error", "error_message": f"Request to OpenWeatherMap timed out for {city}. Please try again later."}
    except requests.exceptions.RequestException as e:
        response_data = {"status": "error", "error_message": f"Error fetching current weather data for {city}: {e}"}
    except KeyError as e:
        response_data = {"status": "error", "error_message": f"Error parsing current weather data for {city}: Missing key: {e}. The API response might be malformed."}
    except Exception as e:
        response_data = {"status": "error", "error_message": f"An unexpected error occurred while getting current weather for {city}: {e}"}

    # This ensures 'last_city_checked_stateful' is updated even if the API returns an error or info.
    if city: # Only update if a valid city was determined for the tool call
        tool_context.state["last_city_checked_stateful"] = city
    
    return response_data

print("`get_current_weather_from_openweather_stateful` custom tool defined.")

In [ ]:
# Custom Tool: Get Weather Summary for a Flexible Date Range (stateful version)
# This tool fetches weather forecasts for a specified city and a date range,
# leveraging the `parse_flexible_date_range` helper to handle diverse date inputs.
# It also uses `tool_context.state` to:
# 1. Read `user_preference_temperature_unit`.
# 2. Remember the `last_city_checked_stateful` and `last_date_expr_stateful` if not provided.
# 3. Update `last_city_checked_stateful` and `last_date_expr_stateful` upon successful lookup.

def get_weather_summary_from_openweather_stateful(
    city: Optional[str], date_expr: Optional[str], tool_context
) -> Dict[str, Any]:
    """
    Summarizes weather data for a given city and flexible date expression from OpenWeatherMap,
    leveraging session state for remembered context and user preferences.
    Updates last_city_checked_stateful and last_date_expr_stateful unconditionally.
    """
    # Retrieve city and date expression from session state if not provided in the tool call
    if not city:
        city = tool_context.state.get("last_city_checked_stateful")
    if not city:
        return {"status": "error", "error_message": "No city specified or remembered for weather forecast. Please provide a city."}
    
    if not date_expr:
        date_expr = tool_context.state.get("last_date_expr_stateful")
    if not date_expr:
        return {"status": "error", "error_message": "No date or date range specified or remembered for weather forecast. Please provide a date or date range."}

    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "metric")
    unit_symbol = "°C" if preferred_unit == "metric" else "°F"

    print(f"--- Tool: get_weather_summary_from_openweather_stateful called for city: {city}, date_expr: {date_expr}, unit: {preferred_unit} ---")

    response_data = {} # Initialize to store tool response
    try:
        date_range = parse_flexible_date_range(date_expr)
        if not date_range:
            return {"status": "error", "error_message": f"I couldn't understand the date expression '{date_expr}'. Please try a more common format like 'tomorrow', 'next week', 'July 1', or 'July 1 to July 5'."}
        
        start, end = date_range
        current_utc_date = datetime.now(timezone.utc).date()
        forecast_limit_days = 5 # Standard for OpenWeatherMap free tier forecast

        if end.date() < current_utc_date: # If the *entire* range is in the past
            response_data = {"status": "info", "report": f"I can only provide forecasts for current or future dates. '{date_expr}' appears to be in the past. If you meant a future date, please specify it."}
        elif (start.date() - current_utc_date).days > forecast_limit_days: # If the *start* of range is too far in future
            response_data = {"status": "info", "report": f"I can only provide a detailed forecast for up to {forecast_limit_days} days from today. '{date_expr}' is too far in the future for a precise summary."}
        else: # Date range is valid for forecast or includes current/near future
            url = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={OPEN_WEATHER_API_KEY}&units={preferred_unit}"
            response = requests.get(url, timeout=10)
            response.raise_for_status()
            data = response.json()

            if data["cod"] != "200":
                 response_data = {"status": "error", "error_message": f"OpenWeatherMap API Error for forecast: {data.get('message', 'Unknown error')}"}
            else:
                all_temps = []
                all_descriptions = []

                for entry in data.get("list", []):
                    entry_dt = datetime.fromtimestamp(entry["dt"]).replace(tzinfo=None) # Make naive for comparison
                    if start <= entry_dt <= end: # Compare full datetimes
                        all_temps.append(entry["main"]["temp"])
                        all_descriptions.append(entry["weather"][0]["description"])

                if not all_temps:
                    # If no forecast entries match the specific date range, provide a generic fallback
                    response_data = {"status": "info", "report": f"No specific forecast entries found for {city} for '{date_expr}' within the available forecast data. This can happen for very precise or far-off dates within the 5-day window."}
                else:
                    min_temp = round(min(all_temps), 1)
                    max_temp = round(max(all_temps), 1)
                    avg_temp = round(sum(all_temps) / len(all_temps), 1)

                    weather_type_counts = Counter(all_descriptions)
                    most_common_weather_types = [desc for desc, _ in weather_type_counts.most_common(3)]
                    weather_description_str = ", ".join(most_common_weather_types)

                    report = (
                        f"The weather in {data['city']['name']} from {start.strftime('%B %d')} to {end.strftime('%B %d')} "
                        f"will be generally {weather_description_str}, with an average temperature of {avg_temp}{unit_symbol}. "
                        f"Temperatures will range from {min_temp}{unit_symbol} to {max_temp}{unit_symbol}."
                    )
                    response_data = {"status": "success", "summary": report}

    except requests.exceptions.Timeout:
        response_data = {"status": "error", "error_message": f"Request to OpenWeatherMap timed out for {city} and '{date_expr}'. Please try again later."}
    except requests.exceptions.RequestException as e:
        response_data = {"status": "error", "error_message": f"Error fetching weather forecast data for {city}: {e}"}
    except KeyError as e:
        response_data = {"status": "error", "error_message": f"Error parsing weather forecast data for {city}: Missing key: {e}. The API response might be malformed or an invalid city was provided."}
    except Exception as e:
        response_data = {"status": "error", "error_message": f"An unexpected error occurred while getting weather summary for {city} and '{date_expr}': {e}"}

    if city:
        tool_context.state["last_city_checked_stateful"] = city
    if date_expr:
        tool_context.state["last_date_expr_stateful"] = date_expr
    
    return response_data

print("`get_weather_summary_from_openweather_stateful` custom tool defined.")

In [ ]:
# Weather Agent: Definition
# This agent leverages the two stateful custom tools defined above.
# Its `instruction` is crucial for guiding the agent to:
# - Use the correct tool based on the user's query (current vs. forecast).
# - Utilize session state to remember the last city and date expression if not explicitly provided.
# - Understand and respond to queries about remembered context.
# - Handle tool outcomes (success, error, info) gracefully.
weather_agent = Agent(
    name="weather_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="Provides current weather and weather summaries, using user preferences and conversational context.",
    instruction=(
        "You are a helpful and accurate weather assistant. "
        "Your responses should be clear and concise. "
        
        "**Context Handling (Session State):**\n"
        "1.  **Remembered City:** If the user does not specify a city in a query, *always* try to use the `last_city_checked_stateful` value from `tool_context.state`. If no city is remembered, you *must* ask the user for a city.\n"
        "2.  **Remembered Date:** If the user does not specify a date or date range in a query, *always* try to use the `last_date_expr_stateful` value from `tool_context.state`. If no date is remembered, you *must* ask the user for a date or date range.\n"
        "3.  **Temperature Unit Preference:** You are aware that `user_preference_temperature_unit` (either 'metric' or 'imperial') is stored in `tool_context.state`. The weather tools will automatically use this. **DO NOT** explicitly mention temperature units in your response unless the user explicitly asks about or changes their preference.\n"
        "4.  **Responding to Context Queries:** If the user asks explicitly about 'What was the last city I checked?' or 'What date range did I last ask about?', you **MUST directly retrieve and provide the value from `tool_context.state.last_city_checked_stateful` or `tool_context.state.last_date_expr_stateful`**. Do not call a tool for these specific questions; your internal knowledge (from session state) is sufficient.\n"
        "5.  **Re-querying on Context Change:** If a user's preference or a key context (like temperature unit) changes during a conversation, and they re-ask a previous type of question, you **MUST re-execute the appropriate tool** to provide an updated response based on the new context.\n"
        
        "**Tool Usage:**\n"
        "1.  When the user asks for the **current weather** in a city (e.g., 'What's the weather in London?'), you MUST use the `get_current_weather_from_openweather_stateful` tool.\n"
        "2.  When the user asks for the **weather forecast** for a specific date or date range (e.g., 'What about tomorrow?', 'weather next week?', 'July 1 to July 5?', 'next 5 days?'), you MUST use the `get_weather_summary_from_openweather_stateful` tool.\n"
        "    *   This tool can handle flexible date expressions like 'this weekend', 'next week', 'last week', 'first week of July', 'tomorrow', 'in 3 days', 'next 5 days', as well as specific dates like '2025-07-01' or 'July 1 to July 5'.\n"
        
        "**Output Handling:**\n"
        "1.  If a tool returns a result with `\"status\": \"success\"`, present the `\"report\"` or `\"summary\"` clearly and concisely.\n"
        "2.  If a tool returns a result with `\"status\": \"error\"`, directly inform the user with the `\"error_message\"` provided by the tool. Do not invent information.\n"
        "3.  If a tool returns a result with `\"status\": \"info\"`, directly provide the `\"report\"` to the user, as these are informative messages (e.g., date out of forecast range).\n"
        "4.  If a query is completely out of scope for weather (e.g., asking for a capital city), politely state your limitations and offer weather assistance.\n"
        "5.  Do not invent information if a tool cannot provide it.\n"
        "6.  Always be friendly and helpful."
    ),
    tools=[get_current_weather_from_openweather_stateful, get_weather_summary_from_openweather_stateful],
)

print("`weather_agent` successfully defined with stateful instructions.")

In [ ]:
# Set up the session and runner for the weather agent
APP_NAME = "wanderwise_app"
USER_ID = "user_001"
SESSION_ID = "weather_demo_session"

# Initialize InMemorySessionService to manage conversational state
session_service = InMemorySessionService()

# Define initial state for the session, including a user preference for temperature unit.
# This state will be accessible by tools via `tool_context.state`.
initial_state = {
    "user_preference_temperature_unit": "imperial" # Start with Fahrenheit
}

# Create a new session, seeding it with the initial state.
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state=initial_state # Pass the initial state here
)

# Initialize InMemoryMemoryService (optional for this specific use case, but good practice if agent needs to write/read more complex long-term memory)
memory_service = InMemoryMemoryService()

# Create the Runner instance, connecting the agent, session service, and memory service.
runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service # Pass memory service to runner
)

print("Session and Runner initialized for weather agent.")

In [ ]:
# Helper function to run and display agent responses
# This asynchronous function handles sending user input to the agent and displaying its final response.
async def ask_weather_agent(runner, prompt: str):
    """
    Sends a prompt to the weather agent and displays its final response.
    Args:
        runner: The ADK Runner instance.
        prompt: The user's input query.
    """
    content = types.Content(role="user", parts=[types.Part(text=prompt)])
    # Use async for to iterate over events from the runner (important for non-blocking execution)
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response():
            # Extract only the text parts from the response
            text_parts = [part.text for part in event.content.parts if hasattr(part, 'text') and part.text]
            # Join all text parts into a single string
            text_response = "\n".join(text_parts)
            display(Markdown(f"**Prompt:** {prompt}"))
            display(Markdown(f"**Agent Response:** {text_response}"))
            break # Exit after the final response is received

print("`ask_weather_agent` helper function defined.")

In [ ]:
# Helper Function: To show current session state
# This function allows us to inspect the `tool_context.state` (which is part of the session state)
# after each interaction, demonstrating how the agent remembers information.
async def show_context():
    """
    Retrieves and prints the current state of the active session.
    """
    # Get the latest session state using the session_service
    current_session = await session_service.get_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    print("\nSession state (per conversation):")
    if current_session.state:
        for k, v in current_session.state.items():
            print(f"  {k}: {v}")
    else:
        print("  <No state stored yet>")

print("`show_context` helper function defined.")

In [ ]:
# Helper function to update session state directly (e.g., changing user preferences)
# This simulates a user setting a preference, which the agent's tools can then read.
async def update_temperature_unit(new_unit: str):
    """
    Updates the temperature unit preference in the session state.
    Args:
        new_unit: The new unit ('metric' for Celsius or 'imperial' for Fahrenheit).
    """
    # Access the session object directly to modify its state
    current_session = session_service.sessions[APP_NAME][USER_ID][SESSION_ID]
    current_session.state["user_preference_temperature_unit"] = new_unit
    print(f"\nUpdated temperature unit preference in session state to: {new_unit}")
    await show_context() # Show updated context immediately

print("`update_temperature_unit` helper function defined.")

In [ ]:
# Example Interactions to demonstrate Conversational Context (Statefulness)

print("====== [starting stateful weather agent interactions] ======\n")

# Define common parameters for our agent interactions.
APP_NAME = "wanderwise_app"
USER_ID = "user_001"
SESSION_ID = "weather_demo_session_stateful" # Use a distinct session ID for this demo

# Initialize InMemorySessionService and create a session with an initial preference.
# This session will persist across turns for this sequence of interactions.
session_service = InMemorySessionService() # Ensure this is initialized here if not global
initial_state = {
    "user_preference_temperature_unit": "imperial" # Start with Fahrenheit
}
await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID,
    state=initial_state
)

# Initialize InMemoryMemoryService (though primarily tool_context.state is used here)
memory_service = InMemoryMemoryService()

# Create the Runner instance, binding it to the persistent session and agent.
runner = Runner(
    agent=weather_agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service
)

print(f"Initialized session '{SESSION_ID}' for user '{USER_ID}'. Initial state:")
await show_context()
time.sleep(1) # Small delay after setup


# --- Test Case 1: Initial query, sets last_city and uses initial unit preference ---
display(Markdown(f"\n**Testing >> Initial query - Current weather, sets city and uses initial unit**"))
await ask_weather_agent(runner, "What's the weather in San Francisco?")
await show_context()
time.sleep(1)


# --- Test Case 2: Subsequent query, relies on remembered city for forecast ---
display(Markdown(f"\n**Testing >> Follow-up Query - Forecast using remembered city**"))
await ask_weather_agent(runner, "And, what will the weather be like for the next 3 days?")
await show_context()
time.sleep(1)


# --- Test Case 3: New city, different date expression, updates context ---
display(Markdown(f"\n**Testing >> New Query - Different city and flexible date expression**"))
await ask_weather_agent(runner, "What's the weather in Paris this weekend?")
await show_context()
time.sleep(1)


# --- Test Case 4: Change preference and re-ask previous query (should use new unit) ---
display(Markdown(f"\n**Testing >> User Changes Preference - Re-ask query with new unit**"))
await update_temperature_unit("metric") # Update preference directly
await ask_weather_agent(runner, "What's the weather in Paris this weekend?") # Re-ask same query
await show_context()
time.sleep(1)


# --- Test Case 5: Ask for weather in a remembered city, explicit future date ---
display(Markdown(f"\n** Testing >> Forecast for remembered city on a specific future date**"))
await ask_weather_agent(runner, "What about London on July 15, 2025?")
await show_context()
time.sleep(1)


# --- Test Case 6: Ask for last remembered city (agent answers from its knowledge of state) ---
display(Markdown(f"\n**Testing >> Ask Agent about its remembered context (last city)**"))
await ask_weather_agent(runner, "What was the last city I checked the weather for?")
await show_context()
time.sleep(1)


# --- Test Case 7: Ask for last remembered date expression (agent answers from its knowledge of state) ---
display(Markdown(f"\n**Testing >> Ask Agent about its remembered context (last date range)**"))
await ask_weather_agent(runner, "What date range did I last ask about?")
await show_context()
time.sleep(1)


# --- Test Case 8: Request for a date too far in the future (beyond 5-day forecast) ---
display(Markdown(f"\n**Testing >> Edge Case - Date too far in the future**"))
await ask_weather_agent(runner, "What will the weather be like in Tokyo in December 2026?")
await show_context()
time.sleep(1)


# --- Test Case 9: Request for a past date (no historical data from API) ---
display(Markdown(f"\n**Testing >> Edge Case - Past date request**"))
await ask_weather_agent(runner, "What was the weather like in New York last week?")
await show_context()
time.sleep(1)


# --- Test Case 10: Invalid City Name (tool error handling) ---
display(Markdown(f"\n**Testing >> Edge Case - Invalid city name**"))
await ask_weather_agent(runner, "What's the weather in NonExistentCity today?")
await show_context()
time.sleep(1)


# --- Test Case 11: Ask for current weather with no prior city remembered and no city specified ---
display(Markdown(f"\n**Testing >> Edge Case - No city given, no city remembered**"))
# Reset session to simulate starting fresh with no context
await session_service.delete_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID, state=initial_state) # Recreate session with initial state
print("\nSession reset to clear previous context for this test.")
await show_context()

await ask_weather_agent(runner, "What's the current weather like?") # Should respond with "No city specified..."
await show_context()
time.sleep(1)


# --- Test Case 12: Ask a non-weather related question (agent should defer or state inability) ---
display(Markdown(f"\n**Testing >> Edge Case - Out of scope query**"))
await ask_weather_agent(runner, "What is the capital of France?")
await show_context()
time.sleep(1)


# --- Test Case 13: Request for "this week" vs. "next week" specifically ---
display(Markdown(f"\n**Testing >> Explicit 'this week' vs 'next week'**"))
await ask_weather_agent(runner, "How's the weather in London this week?")
await show_context()
time.sleep(1)
await ask_weather_agent(runner, "And next week?") # Should remember London and apply to next week
await show_context()
time.sleep(1)


print("\n====== [all stateful weather agent interactions complete] ======")

In [ ]:
# Congratulations 🎉 
# You now have a sophisticated ADK agent that effectively uses conversational context (session state)
# to remember user preferences and past queries. This makes the agent much more user-friendly and capable of
# handling multi-turn dialogues seamlessly.
# You've demonstrated how to:
# - Define stateful custom tools that read from and write to `tool_context.state`.
# - Initialize session state with user preferences.
# - Leverage remembered context for subsequent queries (e.g., city, date range).
# - Allow the agent to directly answer questions about remembered context.
# - Handle cases where context is missing gracefully.
# Remember to ensure your OPEN_WEATHER_API_KEY is correctly set for the tools to function!